In [ ]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from stockpy.neural_network import LSTMClassifier, CNNClassifier
from stockpy.probabilistic import BayesianCNNClassifier, BayesianNNClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
data = pd.read_pickle('../../test/data.pickle')
X = data.drop(['scenario'], axis=1)
y = data['scenario']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle=False)

In [ ]:
# create model instance and fit to training data
predictor = BayesianNNClassifier(hidden_size=64)
predictor.fit(X=X_train, 
              y=y_train,
              lr=0.001,
              patience=10,
              batch_size=8,
              sequence_length=22,
              pretrained=False,
              shuffle=False,
              val_size=0.1,
              scaler_type='minmax',
              epochs=100)

# Append true labels and predicted labels to lists using the score method
true_labels, pred_labels = predictor.score(X_test, y_test)

In [ ]:
# compute f1 score
f1 = f1_score(true_labels, pred_labels, average='weighted')
f1 * 100

In [ ]:
# print confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
# plot the confusion matrix
# Print the confusion matrix using matplotlib
plt.figure(figsize=(10,10))
# add to the plot the f1 score
cm_display = ConfusionMatrixDisplay(cm).plot()
plt.show()